## Importing the libraries

In [1]:
import pydicom
import numpy as np
from PIL import Image
import os

In [3]:
dicom_file = pydicom.dcmread("data/slice.dcm")

In [4]:
dicom_file

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 254
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.826.0.1.3680043.2.1125.1.83176427553122619189183958618878330
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.826.0.1.3680043.2.1143.107.104.103.115.3.0.9.111.124.113
(0002, 0013) Implementation Version Name         SH: 'GDCM 3.0.9'
(0002, 0016) Source Application Entity Title     AE: 'GDCM/ITK 5.3.0'
-------------------------------------------------
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL']
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.826.0.1.3680043.2.1125.1.83176427553122619189183

In [10]:
dicom_array = dicom_file.pixel_array

In [11]:
dicom_array.shape

(512, 512)

## Show Image 1

In [38]:
def normalize_visualize_dicom_1(dcm_file, show=False):
    dicom_file = pydicom.dcmread(dcm_file)
    dicom_array = dicom_file.pixel_array.astype(float)
    normalized_dicom_array = ((np.maximum(dicom_array, 0))/dicom_array.max()) * 255.0
    uint8_image = np.uint8(normalized_dicom_array)

    if show: # if show == True
        pillow_image = Image.fromarray(uint8_image)
        pillow_image.show()

    return uint8_image

In [58]:
image_1 = normalize_visualize_dicom_1('data/slice.dcm', show=True)

## Show Image 2

In [35]:
hounsfield_min = -200 # np.min(dicom_array)
hounsfield_max = 200 # np.max(dicom_array)
hounsfield_range = hounsfield_max - hounsfield_min

In [36]:
dicom_array[dicom_array < hounsfield_min] = hounsfield_min
dicom_array[dicom_array > hounsfield_max] = hounsfield_max
normalized_array_2 = (dicom_array - hounsfield_min) / hounsfield_range
normalized_array_2 *= 255
uint8_image_2 = np.uint8(normalized_array_2)

In [37]:
pillow_image = Image.fromarray(uint8_image_2)
pillow_image.show()

In [2]:
def normalize_visualize_dicom_2(dcm_file, max_v=None, min_v=None, show=True):
    dicom_file = pydicom.dcmread(dcm_file)
    dicom_array = dicom_file.pixel_array

    if max_v: hounsfield_max = max_v
    else: hounsfield_max = np.max(dicom_array)

    if min_v: hounsfield_min = min_v
    else: hounsfield_min = np.min(dicom_array)

    hounsfield_range = hounsfield_max - hounsfield_min

    dicom_array[dicom_array < hounsfield_min] = hounsfield_min
    dicom_array[dicom_array > hounsfield_max] = hounsfield_max
    normalized_array = ((dicom_array - hounsfield_min) / hounsfield_range) * 255
    uint8_image = np.uint8(normalized_array)

    if show:
        pillow_image = Image.fromarray(uint8_image)
        pillow_image.show()
    
    return uint8_image


In [59]:
image_2 = normalize_visualize_dicom_2('data/slice.dcm', show=True, max_v=200, min_v=-200)

In [60]:
dicom_file

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 254
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.826.0.1.3680043.2.1125.1.83176427553122619189183958618878330
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.826.0.1.3680043.2.1143.107.104.103.115.3.0.9.111.124.113
(0002, 0013) Implementation Version Name         SH: 'GDCM 3.0.9'
(0002, 0016) Source Application Entity Title     AE: 'GDCM/ITK 5.3.0'
-------------------------------------------------
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL']
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.826.0.1.3680043.2.1125.1.83176427553122619189183

## Anonymize a dicom file

In [62]:
dicom_file.PatientName = 'pycad'
dicom_file.PatientBirthDate = '01.01.1999'
dicom_file.StudyID = '1234567'
dicom_file.PatientSex = ''

c:\Users\amine\anaconda3\envs\elias\lib\site-packages\pydicom\valuerep.py:290: UserWarning: Invalid value for VR DA: '01.01.1999'.
  warnings.warn(msg)


In [64]:
dicom_file.save_as('data/anonymized_slice.dcm')

In [65]:
new_dicom_file = pydicom.dcmread('data/anonymized_slice.dcm')
new_dicom_file.PatientName

'pycad'

In [66]:
def anonymize_dicom(dcm_file, pn=None, pb=None, st=None, ps=None, save_path=None):
    dicom_file = pydicom(dcm_file)

    if pn: dicom_file.PatientName = pn
    else: dicom_file.PatientName = ''

    if pb: dicom_file.PatientBirthDate = pb
    else: dicom_file.PatientBirthDate = ''

    if st: dicom_file.StudyID = st
    else: dicom_file.StudyID = ''

    if ps: dicom_file.PatientSex = ps
    else: dicom_file.PatientSex = ''

    if save_path:
        dicom_file.save_as(save_path)

    return dicom_file


## Changing tags

In [67]:
def change_tags(dcm_file, ImageType=None, SOPClassUID=None, SOPInstanceUID=None, StudyDate=None,
                SeriesDate=None, ContentDate=None, StudyTime=None, SeriesTime=None,
                ContentTime=None, AccessionNumber=None, Modality=None, Manufacturer=None,
                ReferringPhysicianName=None, PatientName=None, PatientID=None, PatientBirthDate=None,
                PatientSex=None, SliceThickness=None, PatientPosition=None, StudyInstanceUID=None,
                SeriesInstanceUID=None, StudyID=None, SeriesNumber=None, InstanceNumber=None,
                ImagePositionPatient=None, ImageOrientationPatient=None, FrameOfReferenceUID=None,
                PositionReferenceIndicator=None, SamplesPerPixel=None, PhotometricInterpretation=None,
                Rows=None, Columns=None, PixelSpacing=None, BitsAllocated=None, BitsStored=None,
                HighBit=None, PixelRepresentation=None, WindowCenter=None, WindowWidth=None, 
                RescaleIntercept=None, RescaleSlope=None, RescaleType=None, PixelData=None):
    
    
    dicom_file_data = pydicom.dcmread(dcm_file)

    if ImageType: dicom_file_data.ImageType = ImageType
    if SOPClassUID: dicom_file_data.SOPClassUID = SOPClassUID
    if SOPInstanceUID: dicom_file_data.SOPInstanceUID = SOPInstanceUID
    if StudyDate: dicom_file_data.StudyDate = StudyDate
    if SeriesDate: dicom_file_data.SeriesDate = SeriesDate
    if ContentDate: dicom_file_data.ContentDate = ContentDate
    if AccessionNumber: dicom_file_data.AccessionNumber = AccessionNumber
    if Modality: dicom_file_data.Modality = Modality
    if Manufacturer: dicom_file_data.Manufacturer = Manufacturer
    if ReferringPhysicianName: dicom_file_data.ReferringPhysicianName = ReferringPhysicianName
    if PatientName: dicom_file_data.PatientName = PatientName
    if PatientID: dicom_file_data.PatientID = PatientID
    if PatientBirthDate: dicom_file_data.PatientBirthDate = PatientBirthDate
    if PatientSex: dicom_file_data.PatientSex = PatientSex
    if SliceThickness: dicom_file_data.SliceThickness = SliceThickness
    if PatientPosition: dicom_file_data.PatientPosition = PatientPosition
    if StudyInstanceUID: dicom_file_data.StudyInstanceUID = StudyInstanceUID
    if SeriesInstanceUID: dicom_file_data.SeriesInstanceUID = SeriesInstanceUID
    if StudyID: dicom_file_data.StudyID = StudyID
    if SeriesNumber: dicom_file_data.SeriesNumber = SeriesNumber
    if InstanceNumber: dicom_file_data.InstanceNumber = InstanceNumber
    if ImagePositionPatient: dicom_file_data.ImagePositionPatient = ImagePositionPatient
    if ImageOrientationPatient: dicom_file_data.ImageOrientationPatient = ImageOrientationPatient
    if FrameOfReferenceUID: dicom_file_data.FrameOfReferenceUID = FrameOfReferenceUID
    if PositionReferenceIndicator: dicom_file_data.PositionReferenceIndicator = PositionReferenceIndicator
    if SamplesPerPixel: dicom_file_data.SamplesPerPixel = SamplesPerPixel
    if PhotometricInterpretation: dicom_file_data.PhotometricInterpretation = PhotometricInterpretation
    if Rows: dicom_file_data.Rows = Rows
    if Columns: dicom_file_data.Columns = Columns
    if PixelSpacing: dicom_file_data.PixelSpacing = PixelSpacing
    if BitsAllocated: dicom_file_data.BitsAllocated = BitsAllocated
    if BitsStored: dicom_file_data.BitsStored = BitsStored
    if HighBit: dicom_file_data.HighBit = HighBit
    if PixelRepresentation: dicom_file_data.PixelRepresentation = PixelRepresentation
    if WindowCenter: dicom_file_data.WindowCenter = WindowCenter
    if WindowWidth: dicom_file_data.WindowWidth = WindowWidth
    if WindowWidth: dicom_file_data = WindowWidth
    if RescaleIntercept: dicom_file_data.RescaleIntercept = RescaleIntercept
    if RescaleType: dicom_file_data.RescaleType = RescaleType
    if PixelData: dicom_file_data.PixelData = PixelData

    return dicom_file_data


In [68]:
new_dicom_with_tags = change_tags('data/slice.dcm', ImageType='RGB')
new_dicom_with_tags.save_as('data/edited_slice.dcm')

In [69]:
uploaded_dicom = pydicom.dcmread('data/edited_slice.dcm')
uploaded_dicom.ImageType

'RGB'

## Changing the pixel array values

In [71]:
uploaded_dicom.pixel_array

array([[-998, -998, -998, ..., -998, -998, -998],
       [-998, -998, -998, ..., -998, -998, -998],
       [-998, -998, -998, ..., -998, -998, -998],
       ...,
       [-998, -998, -998, ..., -998, -998, -998],
       [-998, -998, -998, ..., -998, -998, -998],
       [-998, -998, -998, ..., -998, -998, -998]], dtype=int16)

In [73]:
type(uploaded_dicom.PixelData)

bytes

In [74]:
new_array = np.zeros((512,512)).tobytes()

In [75]:
new_dicom_pixel_data = change_tags('data/slice.dcm', PixelData=new_array)

In [76]:
new_dicom_pixel_data.pixel_array

c:\Users\amine\anaconda3\envs\elias\lib\site-packages\pydicom\pixel_data_handlers\numpy_handler.py:250: UserWarning: The length of the pixel data in the dataset (2097152 bytes) indicates it contains excess padding. 1572864 bytes will be removed from the end of the data
  warnings.warn(msg)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int16)

## Convert dicom into png/jpg

In [78]:
array = normalize_visualize_dicom_2('data/slice.dcm', max_v=200, min_v=-200, show=False)
img = Image.fromarray(array)
img.save('data/slice.png')

In [7]:
def dicom_to_png(dcm_file, png_path, max_v=None, min_v=None): # png/path/something.dcm
    slice_name = os.path.basename(dcm_file)[:-4]
    array = normalize_visualize_dicom_2(dcm_file, max_v, min_v, show=False)
    img = Image.fromarray(array)
    img.save(f'{png_path}/{slice_name}.png')

In [9]:
dicom_to_png('data/slice.dcm', 'data/test', max_v=200, min_v=-200)

## Convert a group of dicoms into png

In [3]:
from glob import glob

In [11]:
dicom_files_path = glob('data/case_corona_scan/*.dcm')

In [13]:
for dicom_file in dicom_files_path:
    dicom_to_png(dicom_file, 'data/png_folder', 200, -200)

## Convert a group of dicom files into nifti

In [14]:
import dicom2nifti

In [15]:
path_dicom_folder = 'data/case_corona_scan'
dicom2nifti.dicom_series_to_nifti(path_dicom_folder, 'data/case_corona_scan.nii.gz')

{'NII_FILE': 'data/case_corona_scan.nii.gz',
 'NII': <nibabel.nifti1.Nifti1Image at 0x16dff2aa2b0>,
 'MAX_SLICE_INCREMENT': 0.9999990000000001}

## Convert dicom to mp4

In [4]:
import cv2

In [5]:
dicom_files_path = sorted(glob('data/case_corona_scan/*.dcm'))
frame_per_second = 15
frame_size = pydicom.dcmread(dicom_files_path[0]).pixel_array.shape

In [6]:
out = cv2.VideoWriter('data/case_corona_scan.mp4', cv2.VideoWriter_fourcc(*'DIVX'), frame_per_second, frame_size)

In [7]:
for filename in dicom_files_path:
    img = normalize_visualize_dicom_2(filename, max_v=200, min_v=-200, show=False)
    cv2_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    out.write(cv2_img)

out.release()